In [9]:
import xarray as xr
import numpy as np
import requests
import io

### Setup

In [10]:
def test_ensemble_mean(ds, lat, lon, models):

    for var in list(ds.data_vars):
        if var not in ["spatial_ref", "crs"]:
            print(f"Indicator: {var}")

            print("Original Dataset Values:")
            values = (
                ds[var]
                .sel(
                    lat=lat,
                    lon=lon,
                    method="nearest",
                )
                .sel(model=models)
                .values
            )

            print(values)
            print("\n Calculated Ensemble Mean:")
            # skip -9999 when calculating the mean
            values = np.where(values == -9999, np.nan, values)
            print(np.nanmean(values))

            print("Dataset Ensemble Mean:")
            print(
                ds[var]
                .sel(
                    lat=lat,
                    lon=lon,
                    method="nearest",
                )
                .sel(model=13)
                .values
            )

            print("\n\n")

In [11]:
# test will use this location
lat = 65
lon = -142
models = slice(0, 13)  # first 12 models - model 13 is the ensemble mean

# query only first and last timesteps / scenarios of the datacube
times = [0, 54787]
scenarios = [0, 4]

hist_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_indicators_cf&SUBSET=time({times[0]})&SUBSET=scenario({scenarios[0]})&FORMAT=application/netcdf"
proj_url = f"https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_indicators_cf&SUBSET=time({times[1]})&SUBSET=scenario({scenarios[1]})&FORMAT=application/netcdf"

### Historical

In [12]:
with requests.get(hist_url) as response:
    ds_hist = xr.open_dataset(
        io.BytesIO(response.content), decode_cf=True, mask_and_scale=False
    )
ds_hist

<xarray.Dataset> Size: 8MB
Dimensions:      (model: 14, lat: 43, lon: 288)
Coordinates:
  * model        (model) float64 112B 0.0 1.0 2.0 3.0 ... 10.0 11.0 12.0 13.0
  * lat          (lat) float64 344B 90.0 89.06 88.12 87.17 ... 52.3 51.36 50.42
  * lon          (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
Data variables: (12/14)
    crs          |S1 1B ...
    cd           (model, lat, lon) float32 694kB ...
    cdd          (model, lat, lon) int32 694kB ...
    csdi         (model, lat, lon) int32 694kB ...
    cwd          (model, lat, lon) int32 694kB ...
    dw           (model, lat, lon) int32 694kB ...
    ...           ...
    r10mm        (model, lat, lon) int32 694kB ...
    rx1day       (model, lat, lon) int32 694kB ...
    rx5day       (model, lat, lon) int32 694kB ...
    su           (model, lat, lon) int32 694kB ...
    wsdi         (model, lat, lon) int32 694kB ...
    spatial_ref  int32 4B ...
Attributes:
    Conventions:  CF-1.8
    contact:      uaf-snap-data-tools@alaska.edu
    description:  Climate Indicator data from 13 CMIP6 models on a common gri...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    title:        CMIP6 Climate Indicator Data on a Common Grid with Multi-Mo...
    url:          https://uaf-snap.org/

In [13]:
test_ensemble_mean(ds_hist, lat, lon, models)

Indicator: cd
Original Dataset Values:
[  nan -43.7 -46.8 -33.2 -39.7 -51.9 -40.8 -45.  -38.8 -41.2 -42.5 -34.3
 -52.2 -42.5]

 Calculated Ensemble Mean:
-42.50769
Dataset Ensemble Mean:
-42.5



Indicator: cdd
Original Dataset Values:
[25 23 14 16 29 22 13 45 18 12 19 12 38 22]

 Calculated Ensemble Mean:
22.0
Dataset Ensemble Mean:
22



Indicator: csdi
Original Dataset Values:
[-9999    13     7     0     6    12     9     6    18     6     7     0
    22     9]

 Calculated Ensemble Mean:
8.846153846153847
Dataset Ensemble Mean:
9



Indicator: cwd
Original Dataset Values:
[24 12  9  8 13  8  8  9 11 18 23 16  8 13]

 Calculated Ensemble Mean:
12.857142857142858
Dataset Ensemble Mean:
13



Indicator: dw
Original Dataset Values:
[-9999    52    29    11    27    98    50    58    35    45    26    14
    74    43]

 Calculated Ensemble Mean:
43.23076923076923
Dataset Ensemble Mean:
43



Indicator: ftc
Original Dataset Values:
[-9999    47   109    48    85    43    46    53    64 

### Projected

In [14]:
with requests.get(proj_url) as response:
    ds_proj = xr.open_dataset(
        io.BytesIO(response.content), decode_cf=True, mask_and_scale=False
    )
ds_proj

<xarray.Dataset> Size: 8MB
Dimensions:      (model: 14, lat: 43, lon: 288)
Coordinates:
  * model        (model) float64 112B 0.0 1.0 2.0 3.0 ... 10.0 11.0 12.0 13.0
  * lat          (lat) float64 344B 90.0 89.06 88.12 87.17 ... 52.3 51.36 50.42
  * lon          (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
Data variables: (12/14)
    crs          |S1 1B ...
    cd           (model, lat, lon) float32 694kB ...
    cdd          (model, lat, lon) int32 694kB ...
    csdi         (model, lat, lon) int32 694kB ...
    cwd          (model, lat, lon) int32 694kB ...
    dw           (model, lat, lon) int32 694kB ...
    ...           ...
    r10mm        (model, lat, lon) int32 694kB ...
    rx1day       (model, lat, lon) int32 694kB ...
    rx5day       (model, lat, lon) int32 694kB ...
    su           (model, lat, lon) int32 694kB ...
    wsdi         (model, lat, lon) int32 694kB ...
    spatial_ref  int32 4B ...
Attributes:
    Conventions:  CF-1.8
    contact:      uaf-snap-data-tools@alaska.edu
    description:  Climate Indicator data from 13 CMIP6 models on a common gri...
    institution:  Scenarios Network for Alaska and Arctic Planning, Universit...
    source:       CMIP6 model output
    title:        CMIP6 Climate Indicator Data on a Common Grid with Multi-Mo...
    url:          https://uaf-snap.org/

In [15]:
test_ensemble_mean(ds_proj, lat, lon, models)

Indicator: cd
Original Dataset Values:
[-22.5 -27.9   nan -23.3 -26.2 -28.4 -26.8 -32.5 -20.7 -29.1 -21.1 -24.8
 -24.8 -25.7]

 Calculated Ensemble Mean:
-25.676922
Dataset Ensemble Mean:
-25.7



Indicator: cdd
Original Dataset Values:
[   16    19 -9999    23    15    19    18    14    11    18    30    15
    27    19]

 Calculated Ensemble Mean:
18.76923076923077
Dataset Ensemble Mean:
19



Indicator: csdi
Original Dataset Values:
[-9999     0 -9999     0     0     0     0     0     0     0     0     0
     0     0]

 Calculated Ensemble Mean:
0.0
Dataset Ensemble Mean:
0



Indicator: cwd
Original Dataset Values:
[   13     8 -9999     9    15    13    13    11    13    15     8    11
     8    11]

 Calculated Ensemble Mean:
11.384615384615385
Dataset Ensemble Mean:
11



Indicator: dw
Original Dataset Values:
[    0     3 -9999     2     0     4     1     6     0     3     0     0
     0     2]

 Calculated Ensemble Mean:
1.6153846153846154
Dataset Ensemble Mean:
2



Indicator